In [ ]:
#установим соединение с БД по протоколу ODBC
import pandas as pd
import pyodbc 
server = 'HOME\SQLEXPRESS' 
database = 'risks' 
username = 'IT' 
password = '1111' 
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [ ]:
#необходимо указать свою директорию файла
import pandas as pd
rf = pd.ExcelFile (r"C:\RiskFree.xlsx")
RiskFree = pd.read_excel (rf, sheet_name = "Лист1")
RiskFree

In [ ]:
#создадим вспомогательную таблицу bond_yields для расчета доходностей облигаций и нормализуем данные в ней
SELECT [TIME],[Y2O_ASK], [Y2O_BID], [YIELD_ASK],[YIELD_BID],[ISIN]
INTO [risks].[dbo].[bond_yields]
FROM [risks].[dbo].[base_prices];
UPDATE [risks].[dbo].[bond_yields]
SET [Y2O_ASK] = REPLACE([Y2O_ASK], ',', '.');
UPDATE [risks].[dbo].[bond_yields]
SET [Y2O_BID] = REPLACE([Y2O_BID], ',', '.');
UPDATE [risks].[dbo].[bond_yields]
SET [YIELD_ASK] = REPLACE([YIELD_ASK], ',', '.');
UPDATE [risks].[dbo].[bond_yields]
SET [YIELD_BID] = REPLACE([YIELD_BID], ',', '.');
ALTER TABLE [risks].[dbo].[bond_yields] ALTER COLUMN [Y2O_ASK] float;
ALTER TABLE [risks].[dbo].[bond_yields] ALTER COLUMN [Y2O_BID] float;
ALTER TABLE [risks].[dbo].[bond_yields] ALTER COLUMN [YIELD_ASK] float;
ALTER TABLE [risks].[dbo].[bond_yields] ALTER COLUMN [YIELD_BID] float;
SELECT * FROM [risks].[dbo].[bond_yields];
GO

In [ ]:
OurISIN=input('введите ISIN облигации ')

In [ ]:
#нужно изменить isin облигации вручную, чтобы код исполнялся 
qlQuery1= ("""
SELECT [TIME],[Y2O_ASK], [Y2O_BID], [YIELD_ASK],[YIELD_BID] 
FROM [risks].[dbo].[bond_yields] 
WHERE ([ISIN] = 'OurISIN') 
AND ((([Y2O_ASK] != 0) AND ([Y2O_BID] != 0)) 
OR (([YIELD_ASK] != 0) AND ([YIELD_BID] != 0)));
""")
bond_yield = pd.read_sql(SqlQuery1, conn)

In [ ]:
from datetime import datetime #перевод формат даты в тот, с которым можно работать

t = datetime(1900, 1, 1).toordinal() + pd.to_numeric(bond_yield['TIME'].dropna()).astype(int) - 2 
for i in range(0, len(t)):
    t.iloc[i] = datetime.fromordinal(t.iloc[i])
    
bond_yield['date'] = t
bond_yield = bond_yield[['date','Y2O_ASK','Y2O_BID','YIELD_ASK','YIELD_BID']]
bond_yield

In [ ]:
SqlQuery2 = (""" 
SELECT * FROM [risks].[dbo].[bond_information]; 
""")
bonds_info = pd.read_sql(SqlQuery2, conn)
bond_info = bonds_info[bonds_info['ISIN, RegCode, NRDCode'] == OurISIN]
bond_info

In [ ]:
# Выгружаем структуру платежей
from datetime import datetime, timedelta

if bond_info['CouponType'].all() == 'Постоянный' and bond_info['AmortisedMty'].all() == 0:
    
    structure = pd.DataFrame(data={'date': [], 'payment': []})
      
    if bond_info['HaveOffer'].all() == 0:
        
        j = bond_info.loc[:,'EndMtyDate'].astype(object).all()
        i=0
        
        structure.at[i, 'date'] = j
        structure.at[i, 'payment'] = bond_info['coupon'] + bond_info['FaceValue']
        
        while j >= bond_info['BegDistDate'].astype(object).all():
            j -= timedelta(days=365/bond_info.loc[:,'CouponPerYear'].sum())
            i += 1
            structure.at[i, 'date'] = j
            structure.at[i, 'payment'] = bond_info['coupon']
                        
    else:
        offer_date = input('Enter a date in YYYY-MM-DD format: ')
        Offer_date = pd.Timestamp(offer_date)
                              
        j = bond_info['BegDistDate'].astype(object).all()
        i=0     
                
        while j <= Offer_date:
        
            j += timedelta(days=365/bond_info.loc[:,'CouponPerYear'].sum())
            i += 1
            structure.at[i, 'date'] = j
            structure.at[i, 'payment'] = bond_info['coupon']
            
        structure.at[i, 'date'] = Offer_date
        structure.at[i, 'payment'] = bond_info['FaceValue']
else:
    print('Облигация не подходит: тип ее купона не является постоянным или же она является амортизируемой')
    
structure

In [ ]:
# Рассчитываем безрисковую стоимость облигации на основе структуры платежей по облигации и данных о безрисковой доходности
import numpy as np
from math import exp

if bond_info['CouponType'].all() == 'Постоянный' and bond_info['AmortisedMty'].all() == 0:
    #Накладываем условия на тип облигации и купона. Если тип купона не является постоянным или облигация является амортизируемой, 
    #то мы не можем корректно определить структуру платежей по облигации, а тогда невозможен и расчет безрисковой стоимости облигации.
        
    bond_yield ['B_rf'] = bond_yield['YIELD_ASK']
    #Создаем в таблице с торгами столбец, который будет итогом данной программы (будет заполнен безрисковой стоимостью обещанного потока платежей).
    #Пока же заполним его асками.
    
    for i in range(0,len(bond_yield)):
    #используем цикл, чтобы расчеты на каждую из дат торгов
    
        future_payments = structure.where(structure['date'] >= bond_yield.date.iloc[i]).dropna(how='all')
        #Для каждой даты торгов создаем таблицу со структурой предстоящих платежей (без тех, что уже были осуществлены).
                
        future_payments['s'] = round((future_payments['date'] - bond_yield.date.iloc[i])/ np.timedelta64(1, 'D')  / 365, 3)
        #Создаем столбец, показывающий, сколько лет осталось от i-той даты да каждого из платежей. Для формулы из задания это показатель s.
                      
        rf_rates = RiskFree.iloc[(RiskFree['Дата']-bond_yield.date.iloc[i]).abs().argsort()[:1]].copy()
        #На каждую дату торгов (не нее же или на ближайшую к ней) отбираем кривую безрисковой доходности.
                
        rf_rates = rf_rates.melt(id_vars = 'Дата', value_vars=tuple(rf_rates.columns[1:]))
        #Транспонируем таблицу с данными о безрисковой доходности на эту дату.        
        
        rf_rates['variable'] = rf_rates.variable.astype(float)
        #В результате транспонирования строка-заголовков с сроками теперь стала столбцом с именем 'variable'. Убедимся, что столбец имеет числовой тип данных.
        #Также отметим, что столбец со значениями ставок называется теперь 'value'.
                
        def closest_tenors(value):
            exact_tenor = rf_rates[rf_rates.variable == value]
            if exact_tenor.empty:
                lower_value = rf_rates[rf_rates.variable < value][-1:]
                upper_value = rf_rates[rf_rates.variable > value][:1]
                neighbours = lower_value.append(upper_value)
                return neighbours
            else:
                return exact_tenor  
             #Столкнулись с проблемой, что безрисковые ставки есть не на любой срок. Будем решать проблему с помощью линейной интерполяции ставок.
        #Для линейной интерполяции нам необходимо для каждого срока до платежа ближайшие (один больше, другой меньше) значения сроков безрисковой доходности.
        #Используем "Функцию Максима Соломатина", которая для каждого срока платежа из таблицы с безрисковыми ставками выбирает информацию
        #о 2 ближайших (один больше, другой меньше) сроков безрисковой доходности или же об одном сроке, если интересуемый срок есть в таблице безрисковой доходности.
        
        
        future_payments['d_s'] = future_payments['payment']
        future_payments['discounted_payments'] = future_payments['payment']
        #В таблице с предстоящими платежеми создадим столбцы, которые впоследствии будут заполняться коэффициентами дисконтирования и дисконтированными платежами.
            
        for j in range(0, len(future_payments)):
        #Для каждого платежа:
            closest_tenor = closest_tenors(future_payments.s.iloc[j])
            #Сначала применяем описанную выше функцию. 

            if closest_tenor.shape[0] == 1:
                rf_rate = closest_tenor.value.iloc[0]
                #Если интересуемый срок платежа есть в таблице безрисковой доходности, то
                #никакой интерполяции применять не надо. Просто берем ставку, соответствующую данному сроку.
            else:
                rf_rate = closest_tenor.value.iloc[0] + ((closest_tenor.value.iloc[1] - closest_tenor.value.iloc[0]) * (future_payments.s.iloc[j] - closest_tenor.variable.iloc[0]) / (closest_tenor.variable.iloc[1] - closest_tenor.variable.iloc[0]))
                #Если bнтересуемого срока платежа нет в таблице безрисковой доходности, то для расчета безрисковой ставки применяем  
                #линейную интерполяцию на основе 2 ближайших сроков из данных о бескупонной доходности, между которыми заключен интересуемый срок
                
            future_payments.d_s.iloc[j] = exp(-rf_rate/100 * future_payments.s.iloc[j])
            #Считаем столбец с коэффициентами дисконтирования (по формуле из задания).
            future_payments.discounted_payments.iloc[j] = future_payments.payment.iloc[j] * future_payments.d_s.iloc[j]
            #Создаем столбец с дисконтированным потоком платежей. Умножаем платежи на коэффициенты дисконтирования.

        bond_yield.B_rf.iloc[i] = future_payments.discounted_payments.sum()
        #Итого: Для каждой даты торгов мы строим таблицу с предстоящими платежами и их дисконтируем по безрисковой ставке.
        #Сложим все дисконтированные платежи. Получим для каждой даты торгов безрисковую стоимость обещанного потока платежей.
       
else:
    print('Облигация не подходит: тип ее купона не является постоянным или же она является амортизируемой')
    
bond_yield

In [ ]:
import scipy.optimize as optimize

if bond_info['CouponType'].all() == 'Постоянный' and bond_info['AmortisedMty'].all() == 0:
    #Накладываем условия на тип облигации и купона. Если тип купона не является постоянным или облигация является амортизируемой, 
    #то мы не можем корректно определить структуру платежей по облигации, а тогда невозможен и расчет безрисковой стоимости облигации.
        
    bond_yield ['ytm_rf'] = bond_yield['YIELD_ASK']
    #Создаем в таблице с торгами столбец, который будет итогом данной программы (будет заполнен безрисковой стоимостью обещанного потока платежей).
    #Пока же заполним его асками.
    
    for i in range(0,len(bond_yield)):
    #используем цикл, чтобы расчеты на каждую из дат торгов
    
        future_payments = structure.where(structure['date'] >= bond_yield.date.iloc[i]).dropna(how='all')
        #Для каждой даты торгов создаем таблицу со структурой предстоящих платежей (без тех, что уже были осуществлены).
                
        future_payments['s'] = round((future_payments['date'] - bond_yield.date.iloc[i])/ np.timedelta64(1, 'D')  / 365, 3)
        #Создаем столбец, показывающий, сколько лет осталось от i-той даты да каждого из платежей.
                      
        def bond_ytm(B_rf, freq, guess=0.05): 
            ytm_func = lambda y : sum([future_payments.payment.iloc[fp_ytm]/(1+y)**future_payments.s.iloc[fp_ytm] for fp_ytm in range(0, len(future_payments))]) - B_rf
            return optimize.newton(ytm_func, guess)
        #Адаптируем код, находящийся: https://github.com/jamesmawm/Mastering-Python-for-Finance-source-codes/blob/master/B03898_05_Codes/bond_ytm.py
        #Код считает доходность к погашению облигации. Код выдаст такую безрисковую ytm, что если по ней дисконтировать оставшиеся 
        #на определенную дату торгов платежи получится посчитанная нами выше безрисковая стоимость облигации на эту дату.
        #Выше мы для каждой даты торгов получили структуру оставшихся платежей. Для каждого платежа нашли срок от той даты торгов, для которой строилась структура будущих платежей.
        #В пункте 3.с. мы получили безрисковую стоимость облигации на каждую дату торгов. Вносим эту информацию в функцию для расчеты ytm.
                
        if __name__ == "__main__":
            ytm = bond_ytm(bond_yield.B_rf.iloc[i], bond_info.CouponPerYear.iloc[0])
            #Применим функцию.

        bond_yield.ytm_rf.iloc[i] = ytm
        #Запишем результат в таблицу. Получаем для каждой даты торгов значение безрисковой ytm.
        
else:
    print('Облигация не подходит: тип ее купона не является постоянным или же она является амортизируемой')
bond_yield['ytm_rf']=bond_yield['ytm_rf']*100
bond_yield

In [ ]:
for i in range(0, len(bond_yield.date)):
    bond_yield.date.iloc[i] = (bond_yield.date.iloc[i]).date()
import matplotlib.pyplot as plt

if bond_info['CouponType'].all() == 'Постоянный' and bond_info['AmortisedMty'].all() == 0:
                    
    if bond_info['HaveOffer'].all() == 0:
        bond_yield['ask-ytm_rf']=bond_yield['YIELD_ASK']-bond_yield['ytm_rf']
        bond_yield['bid-ytm_rf']=bond_yield['YIELD_BID']-bond_yield['ytm_rf']
        bond_yield.dropna(subset=['YIELD_ASK', 'YIELD_BID']).sort_values(by = 'date')
                
        fig, ax = plt.subplots()
        ax.plot(bond_yield.date, bond_yield['ask-ytm_rf'], label='ask-ytm_rf')
        ax.plot(bond_yield.date, bond_yield['bid-ytm_rf'], label='bid-ytm_rf')
        ax.axis('equal')
        ax.set(xlabel='Время', ylabel='Спрэд, в п.п.')
        plt.title('Динамика спрэдов между обычными и безрисковыми доходностями к погашению по облигации '+OurISIN)
        leg = ax.legend()
        fig.autofmt_xdate()
        plt.show()
                       
    else:
        bond_yield['ask-yto_rf']=bond_yield['Y2O_ASK']-bond_yield['ytm_rf']
        bond_yield['bid-yto_rf']=bond_yield['Y2O_BID']-bond_yield['ytm_rf']
        bond_yield.dropna(subset=['Y2O_ASK', 'Y2O_BID']).sort_values(by = 'date')
        
        fig, ax = plt.subplots()
        ax.plot(bond_yield.date, bond_yield['ask-yto_rf'], label='ask-yto_rf')
        ax.plot(bond_yield.date, bond_yield['bid-yto_rf'], label='bid-yto_rf')
        ax.axis('equal')
        ax.set(xlabel='Время', ylabel='Спрэд, в п.п.')
        plt.title('Динамика спрэдов между обычными и безрисковыми доходностями к оферте по облигации '+OurISIN)
        leg = ax.legend()
        fig.autofmt_xdate()
        plt.show()
        
else:
    print('Облигация не подходит: тип ее купона не является постоянным или же она является амортизируемой')

In [ ]:
import matplotlib.pyplot as plt

if bond_info['CouponType'].all() == 'Постоянный' and bond_info['AmortisedMty'].all() == 0:
                    
    if bond_info['HaveOffer'].all() == 0:
        
        plt.hist(bond_yield['ask-ytm_rf'], 25, facecolor='green')
        plt.xlabel('Спрэд, в п.п.')
        plt.ylabel('Количество')
        plt.title('Гистограмма спрэда между доходностями к погашению по ask и безрисковой по облигации '+OurISIN)
        plt.grid(True)
        plt.show()
        
        plt.hist(bond_yield['bid-ytm_rf'], 25, facecolor='yellow')
        plt.xlabel('Спрэд, в п.п.')
        plt.ylabel('Количество')
        plt.title('Гистограмма спрэда между доходностями к погашению по bid и безрисковой по облигации '+OurISIN)
        plt.grid(True)
        plt.show()
                                       
    else:
        plt.hist(bond_yield['ask-yto_rf'], 25, facecolor='green')
        plt.xlabel('Спрэд, в п.п.')
        plt.ylabel('Количество')
        plt.title('Гистограмма спрэда между доходностями к оферте по ask и безрисковой по облигации '+OurISIN)
        plt.grid(True)
        plt.show()
        
        plt.hist(bond_yield['bid-yto_rf'], 25, facecolor='yellow')
        plt.xlabel('Спрэд, в п.п.')
        plt.ylabel('Количество')
        plt.title('Гистограмма спрэда между доходностями к оферте по bid и безрисковой по облигации '+OurISIN)
        plt.grid(True)
        plt.show()
        
else:
    print('Облигация не подходит: тип ее купона не является постоянным или же она является амортизируемой')

In [ ]:
for i in range(0, len(structure.date)):
    structure.date.iloc[i] = (structure.date.iloc[i]).date()
structure

In [ ]:
# Выгружаем таблицу и указываем путь, куда нужно выгрузить таблицу в формате csv. Если пользователь не захочет придумывать свое название таблицы и свою директорию, то он должен просто нажать ENTER, ничего не вводя.
TableTitle=input("Введите название таблицы (или будет использоваться название по умолчанию) и нажмите ENTER: ")
TablePath = input("Введите директорию, где Вы хотите, чтобы таблица была сохранена,и нажмите ENTER: ")

In [ ]:
if TableTitle == "":
    TableTitle = "Структура_платежей_"+OurISIN+"_"+bond_info.IssuerName.all()
if TablePath == "":
    TablePath = "C:\ProgramData\Anaconda3\Scripts"
structure.to_csv(TablePath+"\\"+TableTitle+'.csv')

In [ ]:
#Выгружаем график с динамикой спрэдов по облигации. Предлагаем пользователя ввести название графика и директорию, куда он хочет его выгрузить в формате png. Если пользователь не захочет придумывать свое название графика и свою директорию, то он должен просто нажать ENTER, ничего не вводя.
ImageTitle=input("Введите название графика (или будет использоваться название по умолчанию): ")
ImagePath = input("Введите директорию, где Вы хотите, чтобы график был сохранен.")

In [ ]:
if ImageTitle == "":
    ImageTitle = 'Динамика_спрэдов_доходностей_'+OurISIN
if ImagePath == "":
    ImagePath = "C:\ProgramData\Anaconda3\Scripts"
    
if bond_info['CouponType'].all() == 'Постоянный' and bond_info['AmortisedMty'].all() == 0:                
    if bond_info['HaveOffer'].all() == 0:                        
        fig, ax = plt.subplots()
        ax.plot(bond_yield.date, bond_yield['ask-ytm_rf'], label='ask-ytm_rf')
        ax.plot(bond_yield.date, bond_yield['bid-ytm_rf'], label='bid-ytm_rf')
        ax.axis('equal')
        ax.set(xlabel='Время', ylabel='Спрэд, в п.п.')
        plt.title('Динамика спрэдов между обычными и безрисковыми доходностями к погашению по облигации '+OurISIN)
        leg = ax.legend()
        fig.autofmt_xdate()
        plt.savefig(ImagePath+'\\'+ImageTitle+'.png', bbox_inches='tight')
        
    else:              
        fig, ax = plt.subplots()
        ax.plot(bond_yield.date, bond_yield['ask-yto_rf'], label='ask-yto_rf')
        ax.plot(bond_yield.date, bond_yield['bid-yto_rf'], label='bid-yto_rf')
        ax.axis('equal')
        ax.set(xlabel='Время', ylabel='Спрэд, в п.п.')
        plt.title('Динамика спрэдов между обычными и безрисковыми доходностями к оферте по облигации '+OurISIN)
        leg = ax.legend()
        fig.autofmt_xdate()
        plt.savefig(ImagePath+'\\'+ImageTitle+'.png', bbox_inches='tight')
       
else:
    print('Облигация не подходит: тип ее купона не является постоянным или же она является амортизируемой')

In [ ]:
# выгружаем отчет. Предлагаем пользователю задать название отчета и директорию выгрузки. Для вывода отчета необходимо подключить weasyprint и pycairo и разместить базовый файл отчета в удобной директории.
ReportTitle=input("Введите название отчета (или будет использоваться название по умолчанию): ")
ReportPath = input("Введите директорию, где Вы хотите, чтобы отчет был сохранен.")

In [ ]:
if ReportTitle == "":
    ReportTitle = 'Отчет_'+OurISIN+"_доходности_и_спрэды"
if ReportPath == "":
    ReportPath = "C:\ProgramData\Anaconda3\Scripts"

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML, Document
import os
                    
if bond_info['HaveOffer'].all() == 0:
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template(r"C:\report.html")
    template_vars = {
        "Description" : "Данный отчет содержит в себе информацию по дням торгов о bid и ask доходностях к погашению у облигации "+OurISIN+", не содежащей оферту, эмитента "+bond_info.IssuerName.all()+", а также график, отражающий динамику спрэдов между этими доходностями и безрисковой доходностью к погашению.",
        "TableDesc": "Данная таблица отчет содержит данные по дням торгов о bid и ask доходностях к погашению у облигации "+OurISIN+".", 
        "title" : "Отчет", 
        "Table": bond_yield[['date','YIELD_ASK','YIELD_BID']].to_html(),
        "ImageDesc": "Данный график демонстрирует динамику спрэдов между  bid и ask доходностями к погашению и безрисковой по облигации "+OurISIN+".",   
        "Image": os.path.abspath(ImagePath+'\\'+ImageTitle+'.png')}
else:
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template(r"C:\report.html")
    template_vars = {
        "Description" : "Данный отчет содержит в себе информацию по дням торгов о bid и ask доходностях к оферте у облигации "+OurISIN+" эмитента "+bond_info.IssuerName.all()+", а также график, отражающий динамику спрэдов между этими доходностями и безрисковой доходностью к оферте.",
        "TableDesc": "Данная таблица отчет содержит данные по дням торгов о bid и ask доходностях к оферте у облигации "+OurISIN+".",
        "title" : "Отчет", 
        "Table": bond_yield[['date','Y2O_ASK','Y2O_BID']].to_html(),
        "ImageDesc": "Данный график демонстрирует динамику спрэдов между  bid и ask доходностями к оферте и безрисковой по облигации "+OurISIN+".",    
        "Image": os.path.abspath(ImagePath+'\\'+ImageTitle+'.png')}

html_out = template.render(template_vars, presentational_hints=True)
html_ = HTML(string=html_out)
html_.write_pdf(ReportPath+"\\"+ReportTitle+".pdf",stylesheets=["style.css"], presentational_hints=True)